In [11]:
!pip3 install --upgrade pip
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip3 install transformers pymupdf pytesseract pillow

/Users/mac/.zshenv:1: command not found: 0x0:0x0
/Users/mac/.zshenv:1: command not found: 0x0:0x0
Looking in indexes: https://download.pytorch.org/whl/cpu
ERROR: Could not find a version that satisfies the requirement torch (from versions: none)
ERROR: No matching distribution found for torch
/Users/mac/.zshenv:1: command not found: 0x0:0x0


In [3]:
!python3.10 -m pip install --upgrade pip

/Users/mac/.zshenv:1: command not found: 0x0:0x0


In [4]:
!python3.10 -m pip install torch torchvision torchaudio

/Users/mac/.zshenv:1: command not found: 0x0:0x0
  Using cached torch-2.2.2-cp310-none-macosx_10_9_x86_64.whl.metadata (25 kB)
  Using cached torchvision-0.17.2-cp310-cp310-macosx_10_13_x86_64.whl.metadata (6.6 kB)
  Using cached torchaudio-2.2.2-cp310-cp310-macosx_10_13_x86_64.whl.metadata (6.4 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)
  Using cached numpy-2.2.6-cp310-cp310-macosx_14_0_x86_64.whl.metadata (62 kB)
  Using cached pillow-11.2.1-cp310-cp310-macosx_10_10_x86_64.whl.metadata (8.9 kB)
  Using cached MarkupSafe-3.0.2-cp310-cp310-macosx_10_9_universal2.whl.metadata (4.0 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.8/150.8 MB 2.4 M

In [5]:
!python3.10 -m pip install transformers pymupdf pytesseract pillow

/Users/mac/.zshenv:1: command not found: 0x0:0x0
  Using cached transformers-4.52.4-py3-none-any.whl.metadata (38 kB)
  Using cached pymupdf-1.26.0-cp39-abi3-macosx_10_9_x86_64.whl.metadata (3.4 kB)
  Using cached pytesseract-0.3.13-py3-none-any.whl.metadata (11 kB)
  Using cached huggingface_hub-0.32.4-py3-none-any.whl.metadata (14 kB)
  Using cached PyYAML-6.0.2-cp310-cp310-macosx_10_9_x86_64.whl.metadata (2.1 kB)
  Using cached regex-2024.11.6-cp310-cp310-macosx_10_9_x86_64.whl.metadata (40 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-macosx_10_12_x86_64.whl.metadata (6.8 kB)
  Using cached safetensors-0.5.3-cp38-abi3-macosx_10_12_x86_64.whl.metadata (3.8 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached hf_xet-1.1.3-cp37-abi3-macosx_10_12_x86_64.whl.metadata (879 bytes)
  Using cached charset_normalizer-3.4.2-cp310-cp310-macosx_10_9_universal2.whl.metadata (35 kB)
  Using cached idna-3

In [8]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch

In [ ]:
import fitz  # PyMuPDF
import os
import json

In [ ]:
def extract_pdf_content(pdf_path, output_dir="output1"):
    # Create output folder if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

    # Open the PDF
    doc = fitz.open(pdf_path)

    full_text = ""
    image_paths = []
    total_images = 0
    image_captions = []

    for page_num, page in enumerate(doc, start=1):
        # ---- Extract Text ----
        page_text = page.get_text()
        full_text += f"\n--- Page {page_num} ---\n{page_text}"

        # ---- Extract Images ----
        image_list = page.get_images(full=True)
        print(f"Page {page_num} has {len(image_list)} images.")

        for img_index, img in enumerate(image_list):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image_ext = base_image["ext"]

            image_filename = f"page_{page_num}_img_{img_index+1}.{image_ext}"
            image_path = os.path.join(output_dir, image_filename)

            with open(image_path, "wb") as f:
                f.write(image_bytes)

            total_images += 1
            print(f"Saved: {image_path}")

            # Generate caption for the image
            raw_image = Image.open(image_path).convert('RGB')
            inputs = processor(raw_image, return_tensors="pt")
            out = model.generate(**inputs)
            caption = processor.decode(out[0], skip_special_tokens=True)
            image_captions.append({
                "image_path": image_path,
                "caption": caption
            })

    # ---- Save text to a .txt file ----
    text_file = os.path.join(output_dir, "extracted_text.txt")
    with open(text_file, "w", encoding="utf-8") as f:
        f.write(full_text)

    # ---- Save structured metadata to JSON ----
    metadata = {
        "total_pages": len(doc),
        "total_images": total_images,
        "text_file": text_file,
        "image_captions": image_captions
    }

    json_file = os.path.join(output_dir, "summary.json")
    with open(json_file, "w", encoding="utf-8") as f:
        json.dump(metadata, f, indent=4)

    print("\n✅ Extraction complete.")
    print(f"Text saved to: {text_file}")
    print(f"{total_images} images saved to: {output_dir}")
    print(f"Metadata saved to: {json_file}")
    return metadata

# ==== RUN ====
# Replace this with your actual PDF path
extract_pdf_content("pdf2.pdf")


In [ ]:
pdf_path = "pdf2.pdf"
extract_pdf_content(pdf_path)